In [17]:
#import packages
import pandas as pd

In [18]:
#read files
humidity = pd.read_csv('humidity.csv')
temperature = pd.read_csv('temperature.csv')
pressure = pd.read_csv('pressure.csv')
weather_description = pd.read_csv('weather_description.csv')
wind_direction = pd.read_csv('wind_direction.csv')
wind_speed = pd.read_csv('wind_speed.csv')